In [1]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
!pip install -q datasets~=3.2.0 pydantic~=2.10.4
!pip install -q peft~=0.14.0 trl~=0.13.0

# 訓練問答模型

在這個筆記本中，我們將展示如何使用 `transformers` 套件訓練問答模型。我們將使用 `transformers` 套件中的 `SFTTrainer` ([Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/sft_trainer)) 類別來訓練模型。

In [25]:
# import garbage collector
import gc

import pandas as pd

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
)
from datasets import load_dataset
from transformers import (
  pipeline,
)

from typing import Any
from pydantic import BaseModel
from pprint import pprint

import torch

# 載入 PEFT 相關套件
from peft import LoraConfig, TaskType, PeftModel, get_peft_model
# 載入 SFTTrainer 相關套件
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM, setup_chat_format

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

## 下載資料

In [3]:
# The full `train` split, only 1% of dataset
immutable_dataset = load_dataset("philschmid/dolly-15k-oai-style", split="train[:1%]")

### 資料包含什麼？

In [ ]:
# 顯示原始資料中包含的 features 以及筆數
immutable_dataset

In [ ]:
# 檢視資料集中的第一筆資料
pprint(immutable_dataset[0]['messages'])

In [ ]:
# 將 messages 欄位分拆成 user 和 assistant 兩個欄位，方便演示
dataset = immutable_dataset.map(
  lambda x: {
    "user": x["messages"][0],
    "assistant": x["messages"][1],
  }
)
# 將 user 或 assistant 欄位中的 content 長於 512 的部分資料過濾掉
dataset = dataset.filter(
  lambda x: len(x["user"]["content"]) <= 512 and len(x["assistant"]["content"]) <= 512
)
# 將 assistant 欄位中的 content 短於 128 的部分資料過濾掉
dataset = dataset.filter(
  lambda x: len(x["assistant"]["content"]) >= 128
)
# 顯示處理後的資料
dataset

In [ ]:
# 顯示前 first_n_data 筆資料
first_n_data = 3
pd.set_option('display.max_colwidth', None)
pd.DataFrame(dataset.select(range(first_n_data)))

## 訓練設定

### 批次大小 (Batch Size) 和 梯度累積步數 (Gradient Accumulation Steps)

批次大小（batch size）和梯度累積步數（gradient accumulation steps）之間的關係可以簡單地說明如下：

* 批次大小（batch size）：每次訓練迭代中使用的樣本數量。較大的批次大小通常需要更多的內存。
* 梯度累積步數（gradient accumulation steps）：在更新模型權重之前累積梯度的迭代次數。這允許使用較小的批次大小來模擬較大的批次大小。

當內存限制無法直接使用大批次大小時，可以通過梯度累積來實現。例如：

* 如果批次大小是 8，梯度累積步數是 4，這相當於使用批次大小為 32（8 * 4）進行訓練。

這樣可以在內存有限的情況下實現大批次大小的效果。

### 半精度浮點數

半精度訓練（Half-Precision Training）是一種使用16位浮點數（FP16）而不是32位浮點數（FP32）來訓練神經網絡的方法。這種方法的主要優點包括：

* 減少內存使用：FP16數據類型佔用的內存比FP32少一半，允許在相同的硬件上訓練更大的模型或使用更大的批次大小。
* 加速計算：許多現代GPU對FP16計算進行了優化，可以更快地執行FP16運算，從而加速訓練過程。
* 節省帶寬：減少內存和帶寬的使用，有助於提高數據傳輸效率。

BFP16 (Brain Floating Point 16)
BFP16 是一種 16 位浮點數格式，主要由 Google 用於其 TPU（Tensor Processing Unit）。BFP16 的優點是它保留了與 FP32 相同的指數範圍，但尾數精度較低，這在某些情況下可以提供更好的數值穩定性。

FP16 (Half-Precision Floating Point)
FP16 是一種標準的 16 位浮點數格式，廣泛用於 GPU 加速的深度學習訓練。FP16 的優點是內存佔用少，計算速度快，但指數範圍和尾數精度都比 FP32 小。

![](https://miro.medium.com/v2/0*HapPSei5sok65wcv)

總體來說，半精度訓練可以在不顯著影響模型性能的情況下，提高訓練效率和資源利用率。

In [8]:
# 訓練相關設定
class Config(BaseModel):
  model_name: str = 'microsoft/Phi-3.5-mini-instruct'
  torch_dtype: Any = torch.bfloat16 # 半精度浮點數
  adam_epsilon: float = 1e-4 # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  saved_model_path: str = 'sample_data/saved_encoder_model' # path to save the trained model
  saved_lora_path: str = 'sample_data/saved_lora_model' # path to save the trained LORA model
  batch_size: int = 2 # size of the input batch in training and evaluation
  gradient_accumulation_steps: int = 2 # number of updates steps to accumulate before performing a backward/update pass
  epochs: int = 50 # number of times to iterate over the entire training dataset
  lr: float = 2e-4 # learning rate, controls how fast or slow the model learns
  weight_decay: float = 0.01 # weight decay, helps the model stay simple and avoid overfitting by penalizing large weights.

  # Pipieline 相關設定
  pipeline_name: str = 'text-generation' # pipeline name
  temperature: float = 0.1 # temperature for sampling
  max_new_tokens: int = 125 # 限制最大生成字數
  repetition_penalty: float = 1.5 # 重複機率, 1~2 之間, 1.0 (no penalty), 2.0 (maximum penalty)

  # LORA 相關設定
  rank: int = 128 # rank of the LORA model
  lora_alpha: int = rank * 2 # alpha of the LORA model

if device.type == 'mps': # 方便在 Apple Silicon 上快速測試
  config = Config(
    torch_dtype=torch.float16, # 在 Apple Silicon 若使用預訓練模型 opt-125m 需要使用全精度浮點數，否則會出現錯誤
    epochs=1,
  )
else:
  config = Config()

## 先觀察 Fine-tuning 前的表現

### 載入 Tokenizer

In [ ]:
# 透過預訓練模型取得 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
  config.model_name,
)
# 檢視 Tokenizer，是否存在 padding token 及 padding side 等資訊
pprint(tokenizer)

* 如果沒有定義 `pad_token`，請定義一個 `pad_token`，並將其加入 Tokenizer 中。
* 如果 `padding_side` 不是 `right`，請將其設定為 `right`。

In [ ]:
# Add pad_token to the tokenizer
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
  print('=== 設定 Padding Token ===')
  pprint(tokenizer)
# Make sure padding_side is 'right'
if tokenizer.padding_side != 'right':
  tokenizer.padding_side = 'right'
  print('=== 設定 Padding Side ===')
  pprint(tokenizer)

### 載入預訓練模型

由於 GPU 記憶體有限，我們將使用半精度進行模型 Fine-tuning。這邊需要留意，使用半精度進行 Fine-tuning 時，`TrainingArguments` 中的 `adam_epsilon` 需要設定為 `1e-4`。預設的 `adam_epsilon` 是 `1e-8`，這個值在半精度訓練時會出現問題。

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
  config.model_name,
  torch_dtype=config.torch_dtype,
  # 這個參數用於優化內存使用，減少模型加載時的 CPU 內存佔用，特別是在內存有限的環境中非常有用。
  low_cpu_mem_usage=True,
).to(device)

In [ ]:
pprint(model)

### 詠唱格式化 (Prompt Formatting)

在語言模型中添加特殊標記對於訓練聊天模型至關重要。這些標記被添加在對話中不同角色之間，例如user、assistant 和 system，幫助模型識別對話的結構和流程。這種設置對於使模型在聊天環境中生成連貫且上下文適當的回應是必不可少的。`trl` 中的 `setup_chat_format()` 函數可以輕鬆地為對話式 AI 任務設置模型和分詞器。

In [13]:
# Set up the chat format with default 'chatml' format
if tokenizer.chat_template is None:
  model, tokenizer = setup_chat_format(model, tokenizer)
  print('=== 設定 chat format ===')
  pprint(tokenizer)

先定義我們的詠唱 (Prompt) 格式。為此，我們將創建一個格式化函數。

請注意，這次我們指定 `add_generation_prompt` 為 `True`，表示回應開始的標記。這確保了當模型生成文本時，它會寫出機器人的回應，而不是做一些意想不到的事情，比如繼續用戶的訊息。請記住，聊天模型仍然只是語言模型，它們被訓練來玩文字接龍，而聊天對它們來說只是一種特殊的文本！你需要用適當的控制標記來引導它們，讓它們知道應該做什麼。

In [14]:
def instruction_formatter(x, tokenize):
  return tokenizer.apply_chat_template(
    [x['user']],
    tokenize=tokenize,
    add_generation_prompt=True,
  )

In [ ]:
# tokenize=False 代表不進行 Tokenize，直接回傳原始文字
instruction_formatter(dataset[0], tokenize=False)

In [ ]:
# tokenize=True 代表進行 Tokenize，回傳 Tokenize 後的 ID
instruction_formatter(dataset[0], tokenize=True)

### Fine-tuning 前的表現

In [17]:
# 載入預訓練模型
generator = pipeline(
  task=config.pipeline_name,
  model=model,
  tokenizer=tokenizer,
  device=device,
)

In [ ]:
# 這個步驟可能會花費一些時間，所以我們只處理前 first_n_data 筆資料
first_n_dataset = dataset.select(range(first_n_data))

# 移除 messages 欄位
first_n_dataset = first_n_dataset.remove_columns('messages')

# 透過預訓練模型生成回應，將其新增到 first_n_dataset 的 pt_response 欄位中
first_n_dataset = first_n_dataset.map(
  lambda x: {
    **x,
    "pt_response": generator(
      instruction_formatter(x, tokenize=False),
      temperature=config.temperature,
      max_new_tokens=config.max_new_tokens, # 限制最大生成字數
      repetition_penalty=config.repetition_penalty, # 重複機率
      return_full_text=False, # 只回傳生成的文字
    )[0]["generated_text"],
  },
)

In [ ]:
# 顯示預訓練模型預測結果
pd.set_option('display.max_colwidth', None)
pd.DataFrame(first_n_dataset)

## 訓練模型

隨著 `trl` 的最新版本發布，現在支持流行的指令 (instruction) 和對話 (conversation) 數據集格式。這意味著我們只需要將數據集轉換為支持的格式之一，`trl` 會處理其餘的部分。這些格式包括：

* 指令格式 instruction format

```json
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
```

* 對話格式 conversational format

```json
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
```

我們所準備的資料集恰巧符合對話格式，因此我們可以直接使用它。

In [ ]:
# 顯示單筆方便閱讀
pprint(dataset[0]['messages'])

### LoRA 的訓練策略 - 降維打擊

LoRA（Low-Rank Adaptation）是一種用於訓練大型語言模型的技術，旨在提高訓練效率並減少計算資源的需求。以下是為何需要透過LoRA訓練的一些原因：

降低計算成本：LoRA 通過將模型的權重矩陣分解為低秩矩陣，顯著減少了參數的數量，從而降低了計算成本和內存需求。

加速訓練速度：由於參數數量減少，LoRA 可以加速模型的訓練過程，使得在相同的硬件資源下能夠更快地完成訓練。

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png)

In [ ]:
# 查看預訓練模型可訓練的參數量，其數量相當龐大，所以需要透過 Low Rank Approximation (LORA) 來降低參數量
print('Parameters: {:,}, Trainable Parameters: {:,}'.format(
  model.num_parameters(),
  model.num_parameters(only_trainable=True)))

#### PEFT 配置

`rank`: 降維的維度，這個練習我們採用 128。

`lora_alpha`: 決定小模型的影響程度，也就是說 Alpha 值越高，越容易把大模型既有的能力給覆蓋掉。這個練習我們採用常見的比例為 `rank` 的兩倍．

`target_module`: 要降維的模型層，可以透過 `model.named_parameters` 查看。

In [ ]:
# PEFT 配置
lora_config = LoraConfig(
  task_type=TaskType.CAUSAL_LM,
  r=config.rank,
  lora_alpha=config.lora_alpha,
  # Phi3ForCausalLM need to specify the target_modules beforehand
  target_modules=['qkv_proj'],
)

pprint(lora_config)

#### 取得 PEFT 模型

搭配預訓模型及 PEFT 配置，我們可以取得 PEFT 模型。我們可以觀察受到降維影響的模型層。

In [ ]:
# 取得 PEFT 模型
peft_model = get_peft_model(
  model, # 預訓練模型
  lora_config, # PEFT 配置
)

In [ ]:
pprint(lora_config)

In [ ]:
# 取得 PEFT 模型, 觀察受 PEFT 影響的模型參數
peft_model

#### 調整 PEFT 模型精度

PEFT 模型的精度是 `torch.float32`，我們可以透過 `model.half()` 將其轉換為半精度。

In [ ]:
# 獲取 PERF 模型參數名稱及型態，確認是否使用半精度浮點數
for name, param in peft_model.named_parameters():
  print(f'{name}: {param.dtype}')

In [52]:
if config.torch_dtype == torch.float16 or config.torch_dtype == torch.bfloat16:
  peft_model = peft_model.half() # 轉換為半精度浮點數

In [ ]:
# 獲取 PERF 模型參數名稱及型態，確認是否使用半精度浮點數
for name, param in peft_model.named_parameters():
  print(f'{name}: {param.dtype}')

In [ ]:
# 查看可訓練的參數量
peft_model.print_trainable_parameters()

### 資料校對器 (Data Collator)

In [59]:
def instruction_completion_formatter(x, tokenize: bool = False):
  return tokenizer.apply_chat_template(
    x['messages'],
    tokenize=tokenize,
    add_generation_prompt=False,
  )

In [ ]:
instruction_completion_formatter(dataset[0])

In [62]:
response_template = '<|assistant|>\n'

data_collator = DataCollatorForCompletionOnlyLM(
  tokenizer=tokenizer,
  response_template=response_template,
)

In [ ]:
# 展示 DataCollatorForCompletionOnlyLM 的輸出, 標籤以 -100 表示 padding
batch = data_collator([instruction_completion_formatter(dataset[i], True) for i in range(first_n_data)])
pprint(batch)

### 定義訓練參數

In [ ]:
training_args = SFTConfig(
  output_dir='sample_data/train_output_qa',
  learning_rate=config.lr,
  per_device_train_batch_size=config.batch_size,
  per_device_eval_batch_size=config.batch_size,
  gradient_accumulation_steps=config.gradient_accumulation_steps,
  num_train_epochs=config.epochs,
  weight_decay=config.weight_decay,
  eval_strategy='epoch', # 每個 epoch 評估一次
  save_strategy='epoch', # 每個 epoch 儲存一次
  load_best_model_at_end=True, # 訓練完後載入最佳模型
  report_to='none', # Disable wandb on colab
  adam_epsilon=config.adam_epsilon, # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  packing=False, # Disable packing when using DataCollatorForCompletionOnlyLM
  save_total_limit=5, # 最多儲存 5 個 checkpoints
)

trainer = SFTTrainer(
    model=peft_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    formatting_func=instruction_completion_formatter,
    data_collator=data_collator,
)


### 開始訓練

In [ ]:
# 開始訓練，這可能需要一些時間
trainer.train()

#### 保存 LoRA 模型參數

In [69]:
# 保存 Lora 参数
peft_model.save_pretrained(
  config.saved_lora_path,
  # warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
  save_embedding_layers=True,
)

#### 保存 Tokenizer

In [ ]:
# 保存 Tokenizer
tokenizer.save_pretrained(config.saved_model_path)

## 評估模型

### 載入微調後 Tokenizer

從已經完成訓練的模型取得 Tokenizer，可以留意這個訓練時保存下來的 Tokenizer 仍保有訓練時的設定，包涵 `pad_token` 和 `padding_side`。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
  config.saved_model_path
)


In [ ]:
# 檢視 Tokenizer 是否存在 padding token 及 padding side 等資訊
pprint(tokenizer.pad_token)

In [ ]:

pprint(tokenizer.padding_side)

### 載入微調後模型

In [75]:
ft_model = PeftModel.from_pretrained(
  model, # 預訓練模型
  config.saved_lora_path, # PEFT 模型
)

### Fine-tuning 後的表現

#### 顯示微調模型預測

In [ ]:
input = instruction_formatter(first_n_dataset[0], tokenize=False)
input

In [ ]:
# 將 input 轉換為 Token ID
input_ids = tokenizer(input, return_tensors='pt').to(device)
pprint(input_ids)

In [ ]:
output_ids = ft_model.generate(
  **input_ids,
  temperature=config.temperature,
  max_new_tokens=config.max_new_tokens,
  repetition_penalty=config.repetition_penalty,
)

In [ ]:
output_ids

In [99]:
# 將 output_ids 轉換為文字
output = tokenizer.decode(
  output_ids[0],
  skip_special_tokens=False,
)

In [ ]:
output

In [ ]:
# 只取得生成的文字, 即 <|assistant|>\n 之後的文字
output.split('<|assistant|>')[1].strip()

#### 批次處理模型表現

In [117]:
# 將以上程式碼整理成一個函式，方便我們批次處理資料
def generator(x):
  input = instruction_formatter(x, tokenize=False)
  input_ids = tokenizer(input, return_tensors='pt').to(device)
  output_ids = ft_model.generate(
    **input_ids,
    temperature=config.temperature,
    max_new_tokens=config.max_new_tokens,
    repetition_penalty=config.repetition_penalty,
  )
  output = tokenizer.decode(output_ids[0], skip_special_tokens=False)
  return output.split('<|assistant|>')[1].strip()

In [ ]:
# 透過微調模型生成回應，將其新增到 first_n_dataset 的 ft_response 欄位中
first_n_dataset = first_n_dataset.map(
  lambda x: {
    **x,
    "ft_response": generator(x),
  },
)

In [ ]:
# 顯示微調模型預測結果
pd.set_option('display.max_colwidth', None)
pd.DataFrame(first_n_dataset)

# (Optional) Download files from Colab workspace

In [ ]:
![[ ! -z "${$COLAB_GPU}" ]] && exit  # Continue only if running on Google Colab

!tar cvzf saved_encoder_model.tgz sample_data/saved_encoder_model/
!tar cvzf saved_lora_model.tgz sample_data/saved_lora_model/

In [ ]:
import os
if 'COLAB_GPU' in os.environ:
  from google.colab import files
  files.download('saved_encoder_model.tgz')
  files.download('saved_lora_model.tgz')